In [1]:
import os
import openai
import sys

os.environ["OPENAI_API_KEY"]="***"

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
persist_directory = '/Users/rupeshshivsharan/Downloads/data'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [5]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

## Create a Chatbot that Works on Your Documents

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name="gpt-4", temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

In [7]:
import panel as pn
import param

pn.extension()

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "/Users/rupeshshivsharan/Downloads/data/llama 2 research paper .pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return



cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-com

Traceback (most recent call last):
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/panel/viewable.py", line 478, in _on_msg
 doc.unhold()
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 431, in unhold
 self.trigger_on_change(event)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 408, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/bokeh/util/callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/panel/reactive.py", line 471, in _comm_change
 state._handle_exception(e)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/panel/io/state.py", line 442, in _handle_exception
 raise exception
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/panel/reactive.py", line 469, in _comm_change
 self._schedule_change(doc, comm)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/panel/reactive.py", line 451, in _schedule_change
 self._change_event(doc)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/panel/reactive.py", line 447, in _change_event
 self._process_events(events)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/panel/reactive.py", line 385, in _process_events
 self.param.update(**self_events)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 2278, in update
 restore = self_._update(*args, **kwargs)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 2318, in _update
 self_._batch_call_watchers()
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 2509, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 2471, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/panel/param.py", line 865, in _replace_pane
 new_object = self.eval(self.object)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/panel/param.py", line 824, in eval
 return eval_function_with_deps(function)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/param/parameterized.py", line 162, in eval_function_with_deps
 return function(*args, **kwargs)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/param/depends.py", line 41, in _depends
 return func(*args, **kw)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/param/reactive.py", line 431, in wrapped
 return eval_fn()(*combined_args, **combined_kwargs)
 File "/var/folders/hr/b0d_hpsn2js3z577s4l947bw0000gn/T/ipykernel_19301/987370414.py", line 33, in convchain
 result = self.qa({"question": query, "chat_history": self.chat_history})
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py", line 145, in warning_emitting_wrapper
 return wrapped(*args, **kwargs)
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/langchain/chains/base.py", line 383, in __call__
 return self.invoke(
 File "/Users/rupeshshivsharan/opt/anaconda3/lib/python3.9/site-packages/langchain/chai

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] Image(str, width=400)